# Imports

# USE TENSORBOARD PLEASE CHRIST I CAN"T TELL ALL THE TRIALS RESULTS

In [34]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

import pickle
import psycopg2

# Read in data

In [4]:
with open("../database/secrets", "r") as file:
    secrets = [i.strip('\n') for i in file.readlines()]


def conn_curs():
    """
    makes a connection to the database dont worry these are dummy keys
    """

    connection = psycopg2.connect(dbname=secrets[4], user=secrets[4],
                                  password=secrets[5], host=secrets[6])
    cursor = connection.cursor()
    return connection, cursor

In [7]:
conn, curs = conn_curs()
df = pd.read_sql("SELECT * FROM posts", conn)

# Inspect

In [8]:
df.drop('id', axis=1, inplace=True)

In [9]:
df.shape

(8735, 2)

In [10]:
df.head()

,text,subreddit
0,Using This Subreddit # Rules\n1. [Be excellent...,MovieSuggestions
1,Looking for a movie where a mother/father go i...,MovieSuggestions
2,The Abyss \n\nHi Everyone. I had originally p...,MovieSuggestions
3,"Good plot twist Hi all,\n\nAny suggestions for...",MovieSuggestions
4,featuring characters who have interesting reas...,MovieSuggestions


# Vectorize

In [26]:
le = LabelEncoder()
le.fit(df.subreddit.unique())

LabelEncoder()

In [27]:
le.classes_

array(['AmItheAsshole', 'AskMen', 'AskWomen', 'HailCorporate',
       'KarmaCourt', 'MovieSuggestions', 'OutOfTheLoop', 'ProRevenge',
       'RedditWritesSeinfeld', 'TalesFromRetail', 'TellMeAFact',
       'WritingPrompts', 'askscience', 'bestof', 'bestoflegaladvice',
       'books', 'boringdystopia', 'britishproblems', 'dating_advice',
       'explainlikeimfive', 'learnSQL', 'learnpython', 'newreddits',
       'nosleep', 'patientgamers', 'pcmasterrace', 'philosophy',
       'talesfromtechsupport', 'tifu', 'whowouldwin', 'worldnews'],
      dtype=object)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df.text, le.transform(df.subreddit), test_size=0.33, random_state=42)

In [11]:
vect = TfidfVectorizer(max_df=.95, min_df=80)

In [13]:
vect.fit(X_train)

TfidfVectorizer(max_df=0.95, min_df=80)

In [14]:
train_df = pd.DataFrame(vect.transform(X_train).todense(), columns=vect.get_feature_names())
train_df.head()

,10,100,12,15,19,20,2020,30,50,able,about,above,absolutely,access,accidentally,account,across,act,acting,action,actual,actually,add,added,advice,afraid,after,again,against,age,ago,agreed,ahead,air,aita,alive,all,allow,allowed,almost,alone,along,already,alright,also,although,always,am,amazing,amount,an,and,angry,another,answer,any,...,which,while,white,who,whole,why,wide,wife,will,window,windows,wish,with,within,without,woke,woman,women,won,wonder,wondering,wont,word,words,work,worked,working,works,world,worry,worse,worst,worth,would,wouldn,wouldnt,wp,write,writing,written,wrong,www,x200b,yeah,year,years,yes,yesterday,yet,you,young,your,youre,yourself,youtube,youve
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.210855,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.309228,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.228994,0.0,0.000000,0.0,0.424488,0.0,0.186610,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.061828,0.0,0.074198,0.063777,0.0,0.0,0.000000,0.093773,0.0,0.000000,0.000000,0.114615,...,0.119722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.078576,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.087996,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.085018,0.0,0.056062,0.0,0.103501,0.0,0.0
2,0.0,0.0,0.0,0.016089,0.0,0.0,0.00843,0.008012,0.0,0.006649,0.045256,0.0,0.00809,0.0,0.0,0.007835,0.014694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01837,0.018742,0.022271,0.0,0.0,0.011935,0.0,0.0,0.0,0.0,0.0,0.036116,0.0,0.0,0.0,0.007702,0.014591,0.006297,0.0,0.030574,0.0,0.000000,0.021025,0.0,0.0,0.019934,0.391581,0.0,0.011757,0.007455,0.000000,...,0.029602,0.010442,0.015915,0.009195,0.047044,0.015076,0.017467,0.057806,0.005029,0.0,0.02568,0.0,0.061523,0.0,0.012158,0.008836,0.0,0.0,0.008233,0.0,0.0,0.007885,0.0,0.0,0.066728,0.007252,0.006491,0.0,0.013298,0.009101,0.0,0.0,0.0,0.013312,0.0,0.0,0.000000,0.016441,0.008322,0.0,0.0,0.0,0.0,0.007825,0.0,0.005670,0.0,0.008349,0.0,0.003503,0.0,0.000000,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.314074,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.110083,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0

In [15]:
val_df = pd.DataFrame(vect.transform(X_test).todense(), columns=vect.get_feature_names())
val_df.head()

,10,100,12,15,19,20,2020,30,50,able,about,above,absolutely,access,accidentally,account,across,act,acting,action,actual,actually,add,added,advice,afraid,after,again,against,age,ago,agreed,ahead,air,aita,alive,all,allow,allowed,almost,alone,along,already,alright,also,although,always,am,amazing,amount,an,and,angry,another,answer,any,...,which,while,white,who,whole,why,wide,wife,will,window,windows,wish,with,within,without,woke,woman,women,won,wonder,wondering,wont,word,words,work,worked,working,works,world,worry,worse,worst,worth,would,wouldn,wouldnt,wp,write,writing,written,wrong,www,x200b,yeah,year,years,yes,yesterday,yet,you,young,your,youre,yourself,youtube,youve
0,0.060809,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.068406,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.110788,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.036394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046214,0.0,0.0,0.000000,0.15554,0.0,0.072314,0.163548,0.0,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.058733,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.120732,0.0,0.0,0.0,0.0,0.0,0.0,0.058505,0.000000,0.0,0.0,0.056637,0.051422,0.0,0.0,0.125604,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.051815,0.0,0.0,0.0,0.0,0.107644,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.064375,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.109549,0.106183,0.0,0.080764,0.000000,0.000000,...,0.067783,0.00000,0.0,0.063163,0.000000,0.000000,0.0,0.0,0.069100,0.0,0.0,0.0,0.000000,0.0,0.000000,0.121394,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.081892,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.048134,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.032857,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.071945,0.000000,0.040822,0.0,0.000000,0.0,0.103976,0.0,0.0,0.0,0.0,0.0,0.034961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044395,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.179554,0.0,0.000000,0.000000,0.000000,...,0.085965,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.087635,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.077926,0.072391,0.0,0.0,0.066474,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.077319,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.030523,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.078555,0.0,0.0,0.0,0.035143,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.065027,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.048981,0.00000,0.0,0.037150,0.024006,0.0,0.054777,0.000000,0.088025,...,0.000000,0.04865,0.0,0.000000,0.062625,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.030173,0.0,0.113291,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.055543,0.0,0.0,0.058194,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.082546,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.100517,0.102556,0.0,0.000000,0.0,0.065305,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.000000,0.225547,0.0,0.000000,0.244748,0.000000,...,0.000000,0.00000,0.0,0.201250,0.000000,0.164995,0.0,0.0,0.000000,0.0,0.0,0.0,0.070873,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.072766,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.068344,0.062051,0.0,0.0,0.000000,0.038342,0.0,0.0,0.0,0.0,0.0,0.0


# Model

In [29]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=train_df.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(df.subreddit.nunique(), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(train_df.to_numpy(), y_train, batch_size=32, epochs=30, validation_data=(val_df.to_numpy(), y_test))

Epoch 1/30
183/183 [==============================] - 0s 2ms/step - loss: 2.5034 - accuracy: 0.3330 - val_loss: 1.4352 - val_accuracy: 0.5841
Epoch 2/30
183/183 [==============================] - 0s 2ms/step - loss: 1.0079 - accuracy: 0.7097 - val_loss: 1.0096 - val_accuracy: 0.7038
Epoch 3/30
183/183 [==============================] - 0s 2ms/step - loss: 0.6216 - accuracy: 0.8189 - val_loss: 0.9327 - val_accuracy: 0.7347
Epoch 4/30
183/183 [==============================] - 0s 2ms/step - loss: 0.4345 - accuracy: 0.8783 - val_loss: 0.9166 - val_accuracy: 0.7423
Epoch 5/30
183/183 [==============================] - 0s 2ms/step - loss: 0.3081 - accuracy: 0.9209 - val_loss: 0.9670 - val_accuracy: 0.7395
Epoch 6/30
183/183 [==============================] - 0s 2ms/step - loss: 0.2271 - accuracy: 0.9455 - val_loss: 1.0315 - val_accuracy: 0.7405
Epoch 7/30
183/183 [==============================] - 0s 2ms/step - loss: 0.1666 - accuracy: 0.9600 - val_loss: 1.0898 - val_accuracy: 0.7350
Epoch 

### Early findings: A simple nueral net seems to be competitive with a simple logistic regression. As more the conventional models, out of the box it seems that the models worth considering are SVM, LogReg, SGD. SVM is slow on training and predictions though.

In [43]:
model = Sequential()
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003), input_dim=train_df.shape[1]))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.003)))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.003)))
model.add(Dropout(.2))
model.add(Dense(df.subreddit.nunique(), activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_df.to_numpy(), y_train, batch_size=32, epochs=30, validation_data=(val_df.to_numpy(), y_test))

Epoch 1/30
183/183 [==============================] - 0s 3ms/step - loss: 3.4940 - accuracy: 0.1276 - val_loss: 2.7182 - val_accuracy: 0.2647
Epoch 2/30
183/183 [==============================] - 0s 2ms/step - loss: 2.5532 - accuracy: 0.3043 - val_loss: 2.2465 - val_accuracy: 0.4755
Epoch 3/30
183/183 [==============================] - 0s 2ms/step - loss: 2.2529 - accuracy: 0.4339 - val_loss: 2.0480 - val_accuracy: 0.5473
Epoch 4/30
183/183 [==============================] - 0s 2ms/step - loss: 2.0835 - accuracy: 0.5109 - val_loss: 1.9607 - val_accuracy: 0.5914
Epoch 5/30
183/183 [==============================] - 0s 2ms/step - loss: 1.9683 - accuracy: 0.5550 - val_loss: 1.8842 - val_accuracy: 0.6011
Epoch 6/30
183/183 [==============================] - 0s 2ms/step - loss: 1.8788 - accuracy: 0.5965 - val_loss: 1.8368 - val_accuracy: 0.6243
Epoch 7/30
183/183 [==============================] - 0s 2ms/step - loss: 1.8357 - accuracy: 0.6124 - val_loss: 1.8321 - val_accuracy: 0.6302
Epoch 

# IF YOU WANT TO USE NN PICKLE LABEL ENCODER, REMEMBER TO MAKE A PREDICT FUNCTION FOR WHATEVER MODEL YOU CHOOSE